In [ ]:
import requests
import zipfile
import os
import pycountry
import pandas as pd
import regex as re

#The "download the temperature data" function
def DownloadFile(url, writeLocation):
    if requests.get(url).status_code == 200:
        with open(writeLocation, 'wb') as file:
            file.write(requests.get(url).content)
        return True
    else:
        return f"exited with {requests.get(url).status_code} as error"

def ExtractZip(zipFile, extractPath):
    try:
        with zipfile.ZipFile(zipFile, 'r') as zip_ref:
            zip_ref.extractall(extractPath)
        return extractPath
    except Exception as e:
        return f"Error: {e}"

def GenerateTemperatureMapping(stationsFile):
    STAIDCountry = {}
    reg = r"[\s]*(\d+),([\w\W\s]*),([a-zA-Z]{2})"
    EUList = ['BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE']
    with open(stationsFile) as f:
        lines = f.readlines()
        lines = lines[17:]
        f.close()
    for line in lines:
        m = re.match(reg, line)
        if m and m.group(3) in EUList:
            STAIDCountry[m.group(1)] = pycountry.countries.get(alpha_2=m.group(3)).name

    return STAIDCountry

tempLocatie = 'data/temp'
# if DownloadFile("https://knmi-ecad-assets-prd.s3.amazonaws.com/download/ECA_blend_tg.zip", tempLocatie+'.zip'):
#     ExtractZip(tempLocatie+'.zip', tempLocatie)

GenerateTemperatureMapping(tempLocatie+'/stations.txt')
    


